In [12]:
""" Imports """

import os
import sys
import pandas as pd
from pathlib import Path
from datetime import timezone
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

# Environment setup
import dotenv
dotenv.load_dotenv(".env", override=True) # Defines MONGO_URI

# Local
project_root = Path.cwd().parent
src_path = str(project_root / "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from analysis.stat import RemediationTable
from analysis.fetch import global_latest_scan

In [22]:
""" Read remediations """

FILE = "remediations.csv"

if not os.path.exists(FILE):
    print("No remediations found. Please run the anlaysis.ipynb notebook")
else:
    df = pd.read_csv(FILE)

tab = RemediationTable(df)

# Filter out Chainguard images based on alpine
OMIT_CSV = "omit.csv"

omit_df = pd.read_csv(OMIT_CSV)
for _, row in omit_df.iterrows():
    tab = tab.filter(repository=row["repository"],
                     purge=True)

df = tab._df

In [23]:
"""
Filter DataFrame to long Chainguard remediations.
Desired columns:

    id: CVE identifier
    image: registry/repo:tag
    component: name (version)
    severity: Grype severity
    start_date: date first observed (Can be None)
    end_date: date no longer observed (Can be None)
    delta: difference between start and end date in days up to one decimal
    min_remediated: Similar to delta but resolves missing start and end dates
    status: Current Chainguard advisory status
"""

def get_component(row: pd.Series) -> str:
    if pd.isna(row['component.name']):
        return 'Not Measured'
    return f"{row['component.name']} {row['component.version']}"

cgr_df = df[df["registry"] == "cgr.dev"]
rem_df = pd.DataFrame()

# Create new columns
rem_df["id"] = cgr_df["id"]
rem_df["image"] = cgr_df.apply(lambda x: f"{x['registry']}/{x['repository']}:{x['tag']}", axis=1)
rem_df["component"] = cgr_df.apply(get_component, axis=1)
rem_df["severity"] = cgr_df["severity"]
rem_df["start_date"] = pd.to_datetime(cgr_df["first_seen_at"])
rem_df["end_date"] = pd.to_datetime(cgr_df["remediated_at"])
rem_df["delta_days"] = cgr_df["rtime"] / 24

cgr_tab = RemediationTable(cgr_df)
cgr_edge_cases_df = cgr_tab.resolve_edge_cases()._df
rem_df["min_remediated_days"] = cgr_edge_cases_df["rtime"] / 24

Resolving edge-cases: 100%|██████████| 974/974 [00:00<00:00, 16234.96it/s]


In [24]:
""" Add status column """

url = "https://console-api.enforce.dev/query"

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0",
    "Content-Type": "application/json"
}

query = gql("""
    query Advisories {
      advisories {
        documents {
          id
          advisories {
            id
            aliases
            events {
              type
              timestamp
            }
          }
        }
      }
    }
""")

variables = {
    "excludeDates": False,
    "privateCatalogOrg": "ce2d1984a010471142503340d670612d63ffb9f6"
}

transport = RequestsHTTPTransport(url=url, headers=headers)
client = Client(transport=transport, fetch_schema_from_transport=True)

DOCUMENTS = client.execute(query, variable_values=variables)["advisories"]["documents"]
MAX_DATE = global_latest_scan().replace(tzinfo=timezone.utc)

def get_status(row: str) -> str:
    cve_id = row["id"]
    component = row["component"].split(" ")[0]

    for doc in DOCUMENTS:
        if doc["id"] != component:
           continue
        
        for advisory in doc["advisories"]:
          if (cve_id != advisory["id"]) and (cve_id not in advisory["aliases"]):
            continue
           
          last_event = None
          for event in advisory["events"]:
            ts = pd.to_datetime(event["timestamp"])
            if ts <= MAX_DATE:
                if (last_event is None) or (ts > pd.to_datetime(last_event["timestamp"])):
                  last_event = event
          if last_event is None:
            return "Not Specified"
          return last_event["type"]
    
    return "No Advisory"

rem_df["status"] = rem_df.apply(get_status, axis=1)
rem_df

,id,image,component,severity,start_date,end_date,delta_days,min_remediated_days,status
0,GHSA-xw73-rw38-6vjc,cgr.dev/chainguard/helm:latest,Not Measured,medium,NaT,2024-04-01 19:02:13.324,NaN,4.619596,No Advisory
1,GHSA-4v7x-pqxf-cx7m,cgr.dev/chainguard/helm:latest,golang.org/x/net v0.17.0,medium,2024-04-20 05:01:22.730,2024-04-21 01:01:15.782,0.833253,0.833253,No Advisory
2,GHSA-33c5-9fx5-fvjm,cgr.dev/chainguard/helm:latest,k8s.io/apimachinery v0.29.0,medium,2024-04-25 05:00:46.907,2024-04-26 05:01:03.121,1.000188,1.000188,No Advisory
3,CVE-2024-24787,cgr.dev/chainguard/helm:latest,stdlib go1.22.2,unknown,2024-05-14 06:02:49.500,2024-05-15 19:01:53.512,1.541019,1.541019,No Advisory
4,CVE-2024-24788,cgr.dev/chainguard/helm:latest,stdlib go1.22.2,unknown,2024-05-14 06:02:49.500,2024-05-15 19:01:53.512,1.541019,1.541019,No Advisory
...,...,...,...,...,...,...,...,...,...
1042,CVE-2024-33601,cgr.dev/chainguard/melange:latest,glibc 2.39-r3,unknown,2024-05-14 06:03:46.534,2024-05-15 17:04:00.260,1.458492,1.458492,FIXED
1043,CVE-2024-33602,cgr.dev/chainguard/melange:latest,glibc 2.39-r5,unknown,2024-05-15 17:04:00.260,2024-05-16 05:04:00.491,0.500003,0.500003,FIXED
1044,CVE-2024-33601,cgr.dev/chainguard/melange:latest,glibc 2.39-r5,unknown,2024-05-15 17:04:00.260,2024-05-16 05:04:00.491,0.500003,0.500003,FIXED
1045,CVE-2024-33599,cgr.dev/chainguard/melange:latest,glibc 2.39-r5,unknown,2024-05-15 17:04:00.260,2024-05-16 05:04:00.491,0.500003,0.500003,FIXED


In [25]:
""" Filter DataFrame to long remedations """

FILTER_DAYS_CRITICAL = 7
FILTER_DAYS_OTHER = 14

critical_df = rem_df[rem_df["severity"] == "critical"]
critical_df = critical_df[critical_df["min_remediated_days"] > FILTER_DAYS_CRITICAL]

other_df = rem_df[rem_df["severity"] != "critical"]
other_df = other_df[other_df["min_remediated_days"] > FILTER_DAYS_OTHER]

In [26]:
""" Display all critical long remediations """

critical_df

,id,image,component,severity,start_date,end_date,delta_days,min_remediated_days,status
179,GHSA-24rp-q3w6-vc56,cgr.dev/chainguard/mariadb:latest,Not Measured,critical,NaT,2024-04-06 17:02:16.137,NaN,9.536954,No Advisory


In [27]:
""" Total other remediations """

other_df.shape[0]

51

In [28]:
""" Number of images affected by long remedations (non critical) per unique CVE id."""
print(f"Unique CVE IDs: {len(other_df['id'].unique())}")
other_df.groupby("id")["image"].count()

Unique CVE IDs: 16


id
CVE-2024-24787          1
CVE-2024-24788          1
CVE-2024-2961           5
GHSA-22q4-f5r6-3xqw     5
GHSA-25w4-hfqg-4r52     4
GHSA-2g68-c3qc-8985     1
GHSA-4h8f-2wvx-gg5w     2
GHSA-4v7x-pqxf-cx7m    12
GHSA-84pr-m4jr-85g5     2
GHSA-8xfc-gm6g-vgpv     4
GHSA-f8h5-v2vg-46rr     1
GHSA-h75v-3vvj-5mfj     2
GHSA-jjg7-2v4v-x38h     2
GHSA-m44j-cfrm-g8qc     4
GHSA-pxhw-596r-rwq5     1
GHSA-v435-xc8x-wvr9     4
Name: image, dtype: int64

In [29]:
""" Number of long remediations by severity """

other_df.groupby("severity")["id"].count()

severity
high        2
low         3
medium     34
unknown    12
Name: id, dtype: int64

In [30]:
""" Export data in pretty format """

# Concat all remediations

long_df = pd.concat([critical_df, other_df], axis=0)

# Round delta and min_remediated
long_df["delta_days"] = long_df["delta_days"].round(2)
long_df["min_remediated_days"] = long_df["min_remediated_days"].round(2)

# Clean up nan dates and delta values
long_df[["start_date", "end_date", "delta_days"]] = long_df[["start_date", "end_date", "delta_days"]].astype(str)
long_df.loc[long_df["start_date"] == "NaT", "start_date"] = "Not Observed"
long_df.loc[long_df["end_date"] == "NaT", "end_date"] = "Not Observed"
long_df.loc[long_df["delta_days"] == "nan", "delta_days"] = "NA"

# Save by match
long_df.to_csv("long-remediations-by-match.csv", index=False)

# Save by CVE ID
by_id_df = long_df.groupby(["id"])["min_remediated_days"].mean().reset_index()
by_id_df["min_remediated_days"] = by_id_df["min_remediated_days"].round(2)

by_id_num_matches_df = long_df.groupby(["id"])["image"].count().reset_index()
by_id_df = by_id_df.merge(by_id_num_matches_df, on="id", how="left")
by_id_df = by_id_df.rename({"image": "num_matches"}, axis=1)

by_id_df = by_id_df.merge(long_df[["id", "severity"]], on="id", how="left").groupby(["id"]).first()
by_id_df.to_csv("long-remediations-by-id.csv", index=True)

: 

keycloak:
    - GHSA-f8h5-v2vg-46rr (CVE-2024-2700)
        > published 15 days ago
        > fixed 6 hours ago